In [1]:
import sys

import os

os.environ["CUDA_VISIBLE_DEVICES"]="-1"
import shutil
from typing import Tuple
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow.keras as keras
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer
from dataset import InquiryDataset
import matplotlib.pyplot as plt


In [2]:
def get_bert_details():
    """
    Returns handle encoder and bert model links.
    :return:
    """
    bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'

    map_name_to_handle = {
        'bert_en_uncased_L-12_H-768_A-12':
            'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
        'bert_en_cased_L-12_H-768_A-12':
            'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
        'bert_multi_cased_L-12_H-768_A-12':
            'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
        'small_bert/bert_en_uncased_L-2_H-128_A-2':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
        'small_bert/bert_en_uncased_L-2_H-256_A-4':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
        'small_bert/bert_en_uncased_L-2_H-512_A-8':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
        'small_bert/bert_en_uncased_L-2_H-768_A-12':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
        'small_bert/bert_en_uncased_L-4_H-128_A-2':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
        'small_bert/bert_en_uncased_L-4_H-256_A-4':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
        'small_bert/bert_en_uncased_L-4_H-512_A-8':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
        'small_bert/bert_en_uncased_L-4_H-768_A-12':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
        'small_bert/bert_en_uncased_L-6_H-128_A-2':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
        'small_bert/bert_en_uncased_L-6_H-256_A-4':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
        'small_bert/bert_en_uncased_L-6_H-512_A-8':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
        'small_bert/bert_en_uncased_L-6_H-768_A-12':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
        'small_bert/bert_en_uncased_L-8_H-128_A-2':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
        'small_bert/bert_en_uncased_L-8_H-256_A-4':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
        'small_bert/bert_en_uncased_L-8_H-512_A-8':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
        'small_bert/bert_en_uncased_L-8_H-768_A-12':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
        'small_bert/bert_en_uncased_L-10_H-128_A-2':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
        'small_bert/bert_en_uncased_L-10_H-256_A-4':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
        'small_bert/bert_en_uncased_L-10_H-512_A-8':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
        'small_bert/bert_en_uncased_L-10_H-768_A-12':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
        'small_bert/bert_en_uncased_L-12_H-128_A-2':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
        'small_bert/bert_en_uncased_L-12_H-256_A-4':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
        'small_bert/bert_en_uncased_L-12_H-512_A-8':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
        'small_bert/bert_en_uncased_L-12_H-768_A-12':
            'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
        'albert_en_base':
            'https://tfhub.dev/tensorflow/albert_en_base/2',
        'electra_small':
            'https://tfhub.dev/google/electra_small/2',
        'electra_base':
            'https://tfhub.dev/google/electra_base/2',
        'experts_pubmed':
            'https://tfhub.dev/google/experts/bert/pubmed/2',
        'experts_wiki_books':
            'https://tfhub.dev/google/experts/bert/wiki_books/2',
        'talking-heads_base':
            'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
    }

    map_model_to_preprocess = {
        'bert_en_uncased_L-12_H-768_A-12':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'bert_en_cased_L-12_H-768_A-12':
            'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
        'small_bert/bert_en_uncased_L-2_H-128_A-2':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-2_H-256_A-4':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-2_H-512_A-8':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-2_H-768_A-12':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-4_H-128_A-2':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-4_H-256_A-4':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-4_H-512_A-8':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-4_H-768_A-12':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-6_H-128_A-2':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-6_H-256_A-4':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-6_H-512_A-8':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-6_H-768_A-12':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-8_H-128_A-2':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-8_H-256_A-4':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-8_H-512_A-8':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-8_H-768_A-12':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-10_H-128_A-2':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-10_H-256_A-4':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-10_H-512_A-8':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-10_H-768_A-12':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-12_H-128_A-2':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-12_H-256_A-4':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-12_H-512_A-8':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'small_bert/bert_en_uncased_L-12_H-768_A-12':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'bert_multi_cased_L-12_H-768_A-12':
            'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
        'albert_en_base':
            'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
        'electra_small':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'electra_base':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'experts_pubmed':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'experts_wiki_books':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
        'talking-heads_base':
            'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    }

    tfhub_handle_encoder = map_name_to_handle[bert_model_name]
    tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

    print(f'BERT model selected           : {tfhub_handle_encoder}')
    print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')
    return tfhub_handle_preprocess, tfhub_handle_encoder



In [3]:
class InquiryAnalyzerBERTModel(keras.Model):

    def __init__(self, tfhub_handle_preprocess: str, tfhub_handle_encoder: str, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.input_layer = keras.Input(shape=(), dtype=tf.string, name="INPUT")
        self.preprocess_layer = hub.KerasLayer(tfhub_handle_preprocess, name="PREPROCESS")
        self.encoder_layer = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name="BERT_ENCODER")
        self.dropout = keras.layers.Dropout(0.1, name="DROPOUT")
        self.dense = keras.layers.Dense(10, activation="softmax", name="FINAL_CLASSIFIER")

    def call(self, inputs, training=None, mask=None):
        # y = self.input_layer(inputs)
        y = self.preprocess_layer(inputs)
        y = self.encoder_layer(y)['pooled_output']
        y = self.dropout(y)
        y = self.dense(y)
        return y


In [4]:
class InquiryAnalyzerBERT:
    def __init__(self, model):
        self.model = model
        pass

    def train(self, ds: tf.data.Dataset, epochs: int=100):
        optimizer = InquiryAnalyzerBERT.get_optimizer(ds, epochs)
        loss = InquiryAnalyzerBERT.get_loss()
        metrics = InquiryAnalyzerBERT.get_metrics()
        self.model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
        print(self.model)
        logs = self.model.fit(x=ds, epochs=epochs)
        return logs
    
    @staticmethod
    def get_optimizer(ds: tf.data.Dataset, epochs: int):
        steps_per_epoch = tf.data.experimental.cardinality(ds).numpy()
        num_train_steps = steps_per_epoch * epochs
        # used to increase the learning rate of the first 10% of the dataset
        num_warmup_steps = int(0.1 * num_train_steps)

        init_lr = 3e-5
        optimizer = optimization.create_optimizer(init_lr=init_lr, num_train_steps=num_train_steps,
                                                  num_warmup_steps=num_warmup_steps, optimizer_type="adamw")
        return optimizer
    @staticmethod
    def get_loss():
        loss = tf.keras.losses.BinaryCrossentropy()
        return loss
    @staticmethod
    def get_metrics():
        metrics = tf.metrics.BinaryAccuracy()
        return metrics

In [5]:
def classifierstring(a: np.ndarray):
    result = ["" for i in a]
    for i, el in enumerate(a):
        if a[i, 4] == 1:
            result[i] += "ORDER "
        if a[i, 2] == 1:
            result[i] += "SEARCH "
        if a[i, 3] == 1:
            result[i] += "DELIVERY "
        if a[i, 7] == 1:
            result[i] += "CHECKOUT "
        if a[i, 0] == 1:
            result[i] += "USER INTERACTION NEEDED"
        if a[i, 1] == 1:
            result[i] += "CONTACT"
        if a[i, 8] == 1:
            result[i] += "REQUEST "
        if a[i, 6] == 1:
            result[i] += "FEEDBACK "
        if a[i, 5] == 1:
            result[i] += "WELCOME "
        if a[i, 9] == 1:
            result[i] += "RECOMMENDATION "
    return result


In [6]:
def get_ds(BATCH_SIZE: int, dataset_path: str):
    npdataset = InquiryDataset.get_training_dataset(dataset_path)
    train_examples = tf.convert_to_tensor(npdataset[:, 0], dtype=tf.string)
    train_labels = np.stack(npdataset[:, 1])
    assert train_labels.shape[2] == 10
    train_labels = train_labels.reshape((npdataset[:, 1].shape[0], 10))  # tensorflow can't convert labels easily
    train_labels = tf.convert_to_tensor(train_labels, dtype=tf.int32)
    dataset = tf.data.Dataset.from_tensor_slices((train_examples, train_labels))
    train_ds = dataset.batch(BATCH_SIZE, drop_remainder=False).cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    return train_ds


def train(savepath, epochs=1000, model: keras.Model=None):
    # assert(type(savepath) == str and callback == None or type(callback) == keras.callbacks.ModelCheckpoint)
    BATCH_SIZE = 32
    # lets get the dataset first
    train_ds = get_ds(BATCH_SIZE, "D:\\Documents\\Code\\Stock\\NNS\\inquiryProcessor\\inquiries_dataset.csv", ) # TODO: TYPE IN YOUR GIT PATH
    print(train_ds)
    # TODO: SOLVE THIS LOCAL PATH PROBLEM
    if model == None:
        tfhub_handle_preprocess, tfhub_handle_encoder = get_bert_details()
        model = InquiryAnalyzerBERTModel(tfhub_handle_preprocess, tfhub_handle_encoder)
    analyzer = InquiryAnalyzerBERT(model)
    logs = analyzer.train(ds=train_ds, epochs=epochs)
    model.save(savepath, save_format="tf")  # creates a folder inside NNS/InquiryProcessor\
    return logs


def get_model_from_file(filepath, ds: tf.data.Dataset, epochs: int):
    model = keras.models.load_model(filepath, custom_objects={"AdamWeightDecay": InquiryAnalyzerBERT.get_optimizer(ds, epochs)})
    return model
    

In [8]:
savepath = "InquiryEstimatorBERT_Model"
a,b = get_bert_details()
model = InquiryAnalyzerBERTModel(a,b)
train(savepath=savepath, model=model, epochs=100)

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3
D:\Documents\Code\Stock\NNS\inquiryProcessor\inquiries_dataset.csv


C:\Users\senya\AppData\Local\Temp/ipykernel_12488/3110488194.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  npdataset = InquiryDataset.get_training_dataset(dataset_path)


<PrefetchDataset shapes: ((None,), (None, 10)), types: (tf.string, tf.int32)>
Epoch 1/100
8/8 [==============================] - 119s 14s/step - loss: 0.9171 - binary_accuracy: 0.8922
Epoch 2/100
8/8 [==============================] - 112s 14s/step - loss: 0.8173 - binary_accuracy: 0.8918
Epoch 3/100
8/8 [==============================] - 109s 14s/step - loss: 0.6608 - binary_accuracy: 0.8922
Epoch 4/100
8/8 [==============================] - 113s 14s/step - loss: 0.5103 - binary_accuracy: 0.8922
Epoch 5/100
8/8 [==============================] - 111s 14s/step - loss: 0.3931 - binary_accuracy: 0.8918
Epoch 6/100
8/8 [==============================] - 104s 13s/step - loss: 0.3354 - binary_accuracy: 0.8926
Epoch 7/100
8/8 [==============================] - 106s 13s/step - loss: 0.2985 - binary_accuracy: 0.8930
Epoch 8/100
8/8 [==============================] - 104s 13s/step - loss: 0.2760 - binary_accuracy: 0.8977
Epoch 9/100
8/8 [==============================] - 104s 13s/step - loss: 0

INFO:tensorflow:Assets written to: InquiryEstimatorBERT_Model\assets


INFO:tensorflow:Assets written to: InquiryEstimatorBERT_Model\assets


In [ ]:
model.summary()

Model: "inquiry_analyzer_bert_model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
PREPROCESS (KerasLayer)      multiple                  0         
_________________________________________________________________
BERT_ENCODER (KerasLayer)    multiple                  28763649  
_________________________________________________________________
DROPOUT (Dropout)            multiple                  0         
_________________________________________________________________
FINAL_CLASSIFIER (Dense)     multiple                  5130      
Total params: 28,768,779
Trainable params: 28,768,778
Non-trainable params: 1
_________________________________________________________________


In [10]:
def classify(a: list) -> Tuple[str, np.ndarray]:
    for i in a:
        assert type(i) == str
    result_tensor = model.call(tf.convert_to_tensor(tf.convert_to_tensor(a)))
    result_np = result_tensor.numpy()
    return classifierstring(result_np.round()), result_np.round()

In [11]:
classify(["Hi, I've ordered pizza Chicago a week ago and I would like to say that it was the best pizza I've ever tried in my life! It's gorgeous and delicious as hell. I have no idea how these guys are doing it!"])

(['FEEDBACK '],
 array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]], dtype=float32))

In [9]:
tf.__version__

'2.7.1'